<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/KNN_Classifier_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 67 kB/s 
     |████████████████████████████████| 198 kB 49.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=2fdeee0820dabda98ff63d74328b1c1c51e5002bb2947048110060d141d9941c
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate() 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [7]:
wikiCategory = sc.textFile("wiki-categorylinks-small.csv.bz2")
wikiPages = sc.textFile("WikipediaPagesOneDocPerLine1000LinesSmall.txt.bz2")

In [8]:
wikiCats = wikiCategory.map(lambda x: x.split(",")).map(lambda x: (x[0].replace('"', ''), x[1].replace('"', '')))
validLines = wikiPages.filter(lambda x: 'id' in x and 'url=' in x)

In [9]:
numberOfDocs = wikiPages.count()
numberOfDocs

1000

In [10]:
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6])) 

In [11]:
keyAndText.take(2)

[('431949',
  'Black people and MormonismWhile at least two black men held the priesthood in the early church, from the mid-1800s until 1978, The Church of Jesus Christ of Latter-day Saints (LDS Church) had a policy which prevented most men of black African descent from being ordained to the church\'s lay priesthood. Under the temple and priesthood restrictions before 1978, black members of African descent could not receive the priesthood or participate in temple ordinances besides baptisms for the dead. For a time in the 1960s and 1970s, they were not allowed to perform baptisms for the dead either. For young men and men in the LDS church, holding the priesthood is required to hold leadership roles, perform baptisms, to bless the sacrament, to bless babies and to bless the sick. Since black men could not hold the priesthood, they were excluded from holding leadership roles and performing these rituals. Temple ordinances are necessary for members to receive the endowment and marriage s

In [12]:
def buildArray(listOfIndices):
  returnVal = np.zeros(20000)

  for index in listOfIndices:
    returnVal[index] = int(returnVal[index]) + 1

  mysum = np.sum(returnVal)

  returnVal = np.divide(returnVal, mysum)
  return returnVal


def build_zero_one_array(listOfIndices):
  returnVal = np.zeros (20000)
    
  for index in listOfIndices:
    if returnVal[index] == 0: returnVal[index] = 1
    
  return returnVal


def stringVector(x):
    returnVal = str(x[0])
    for j in x[1]:
        returnVal += ',' + str(j)
    return returnVal


def cousinSim (x,y):
	normA = np.linalg.norm(x)
	normB = np.linalg.norm(y)
	return np.dot(x,y)/(normA*normB)

In [28]:
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))

In [29]:
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

In [33]:
keyAndListOfWords.take(2)

[('431949',
  ['black',
   'people',
   'and',
   'mormonismwhile',
   'at',
   'least',
   'two',
   'black',
   'men',
   'held',
   'the',
   'priesthood',
   'in',
   'the',
   'early',
   'church',
   'from',
   'the',
   'mid',
   's',
   'until',
   'the',
   'church',
   'of',
   'jesus',
   'christ',
   'of',
   'latter',
   'day',
   'saints',
   'lds',
   'church',
   'had',
   'a',
   'policy',
   'which',
   'prevented',
   'most',
   'men',
   'of',
   'black',
   'african',
   'descent',
   'from',
   'being',
   'ordained',
   'to',
   'the',
   'church',
   's',
   'lay',
   'priesthood',
   'under',
   'the',
   'temple',
   'and',
   'priesthood',
   'restrictions',
   'before',
   'black',
   'members',
   'of',
   'african',
   'descent',
   'could',
   'not',
   'receive',
   'the',
   'priesthood',
   'or',
   'participate',
   'in',
   'temple',
   'ordinances',
   'besides',
   'baptisms',
   'for',
   'the',
   'dead',
   'for',
   'a',
   'time',
   'in',
   

In [38]:
allWords = keyAndListOfWords.flatMap(lambda x: [(y, 1) for y in x[1]])

In [39]:
allWords.take(3)

[('black', 1), ('people', 1), ('and', 1)]

In [16]:
#allWords_2d = keyAndListOfWords.map(lambda x: x[1])
#allWords_1d = allWords_2d.flatMap(lambda x:x)
#allWords = allWords_1d.map(lambda x: (x, 1))

In [40]:
allWords.count()

971463

In [41]:
allCounts = allWords.reduceByKey(lambda x, y: x+y)

In [42]:
allCounts.take(3)

[('at', 4810), ('two', 1848), ('priesthood', 78)]

In [43]:
allCounts.count()

49429

In [44]:
topWords = allCounts.top(20000, lambda x: x[1])

In [45]:
print("Top Words in Corpus:", allCounts.top(10, key=lambda x: x[1]))

Top Words in Corpus: [('the', 74530), ('of', 34512), ('and', 28479), ('in', 27758), ('to', 22583), ('a', 21212), ('was', 12160), ('as', 8811), ('for', 8773), ('on', 8435)]


In [46]:
topWordsK = sc.parallelize(range(20000))
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

In [48]:
topWordsK.take(3)

[0, 1, 2]

In [50]:
topWords[3]

('in', 27758)

In [47]:
print("Word Postions in our Feature Matrix. Last 20 words in 20k positions: ", dictionary.top(20, lambda x : x[1]))

Word Postions in our Feature Matrix. Last 20 words in 20k positions:  [('quebecor', 19999), ('poten', 19998), ('kasada', 19997), ('yadnya', 19996), ('drift', 19995), ('iata', 19994), ('satire', 19993), ('expreso', 19992), ('olimpico', 19991), ('auxiliaries', 19990), ('tenses', 19989), ('petherick', 19988), ('stowe', 19987), ('infimum', 19986), ('parramatta', 19985), ('rimpac', 19984), ('hyderabad', 19983), ('cubes', 19982), ('meats', 19981), ('chaat', 19980)]


In [51]:
allWordsWithDocID = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

In [54]:
allDictionaryWords = dictionary.join(allWordsWithDocID)

In [85]:
allWordsWithDocID.take(3)

[('black', '431949'), ('people', '431949'), ('and', '431949')]

In [55]:
allDictionaryWords.take(5)

[('of', (1, '431949')),
 ('of', (1, '431949')),
 ('of', (1, '431949')),
 ('of', (1, '431949')),
 ('of', (1, '431949'))]

In [101]:
allDictionaryWords_dis = allDictionaryWords.distinct()

In [102]:
allDictionaryWords_dis.take(5)

[('two', ('431952', 40)),
 ('two', ('431971', 40)),
 ('two', ('431989', 40)),
 ('two', ('432014', 40)),
 ('two', ('432015', 40))]

In [60]:
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))

In [61]:
justDocAndPos.take(5)

[('431949', 1), ('431949', 1), ('431949', 1), ('431949', 1), ('431949', 1)]

In [62]:
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

In [63]:
allDictionaryWordsInEachDoc.take(2)

[('431952', <pyspark.resultiterable.ResultIterable at 0x7fa12c9aa510>),
 ('431971', <pyspark.resultiterable.ResultIterable at 0x7fa12c9aa850>)]

In [64]:
def buildArray(listOfIndices):
  returnVal = np.zeros(20000)

  for index in listOfIndices:
    returnVal[index] = returnVal[index] + 1

  mysum = np.sum(returnVal)

  returnVal = np.divide(returnVal, mysum)
  return returnVal

In [65]:
buildArray(allDictionaryWordsInEachDoc.take(1)[0][1])

array([0.08850932, 0.02950311, 0.04658385, ..., 0.        , 0.        ,
       0.        ])

In [66]:
len(list(allDictionaryWordsInEachDoc.take(1)[0][1]))

644

In [67]:
# The following line this gets us a set of
# (docID,  [dictionaryPos1, dictionaryPos2, dictionaryPos3...]) pairs
# and converts the dictionary positions to a bag-of-words numpy array...
allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(list(x[1]))))

In [68]:
print(allDocsAsNumpyArrays.take(3))

[('431952', array([0.08850932, 0.02950311, 0.04658385, ..., 0.        , 0.        ,
       0.        ])), ('431971', array([0.08553655, 0.02488336, 0.05132193, ..., 0.        , 0.        ,
       0.        ])), ('431989', array([0.09074244, 0.03574702, 0.03849679, ..., 0.        , 0.        ,
       0.        ]))]


In [69]:
zeroOne = allDocsAsNumpyArrays.map(lambda x: (x[0], np.array([0 if i == 0 else 1 for i in x[1]])))

In [70]:
zeroOne.take(1)

[('431952', array([1, 1, 1, ..., 0, 0, 0]))]

In [71]:
dfArray = zeroOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]

In [72]:
dfArray.take(2)

869

In [73]:
dfArray[4]

922

In [74]:
multiplier = np.full(20000, numberOfDocs)

In [75]:
multiplier[1000]

1000

In [159]:
idfArray = np.log(np.divide(multiplier, dfArray))

In [160]:
allDocsAsNumpyArrays.take(2)

[('431952',
  array([0.08850932, 0.02950311, 0.04658385, ..., 0.        , 0.        ,
         0.        ])),
 ('431971',
  array([0.08553655, 0.02488336, 0.05132193, ..., 0.        , 0.        ,
         0.        ]))]

In [161]:
allDocsAsNumpyArraysTFidf = allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], idfArray)))

In [162]:
allDocsAsNumpyArraysTFidf.take(2)

[('431952',
  array([0.00747631, 0.00246002, 0.00654094, ..., 0.        , 0.        ,
         0.        ])),
 ('431971',
  array([0.0072252 , 0.00207481, 0.00720622, ..., 0.        , 0.        ,
         0.        ]))]

In [163]:
wikiCats.take(1)

[('434042', '1987_debut_albums')]

In [164]:
featuresRDD = wikiCats.join(allDocsAsNumpyArraysTFidf).map(lambda x: (x[1][0], x[1][1]))

In [165]:

wikiCats.join(allDocsAsNumpyArraysTFidf).take(1)

[('456050',
  ('All_articles_with_unsourced_statements',
   array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
          0.        ])))]

In [166]:
featuresRDD = wikiCats.join(allDocsAsNumpyArraysTFidf).map(lambda x: (x[1][0], x[1][1]))

featuresRDD.cache()
featuresRDD.take(10)

[('All_articles_with_unsourced_statements',
  array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
         0.        ])),
 ('Articles_with_inconsistent_citation_formats',
  array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
         0.        ])),
 ('Articles_with_unsourced_statements_from_February_2013',
  array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
         0.        ])),
 ('CS1:_long_volume_value',
  array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
         0.        ])),
 ('CS1_maint:_archived_copy_as_title',
  array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
         0.        ])),
 ('Continued_fractions',
  array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
         0.        ])),
 ('Gamma_and_related_functions',
  array([0.00619833, 0.00281363, 0.00376037, ..., 0.        , 0.        ,
         0.        ])),
 ('CS1_Spanish-language_sources_(es)'

In [93]:
def zero_check(x):
    if (x == 0): 
        return 0.0001
    else:
        return x

In [99]:
def getPrediction (textInput, k):
    # Create an RDD out of the textIput
    myDoc = sc.parallelize (('', textInput))

    # Flat map the text to (word, 1) pair for each word in the doc
    wordsInThatDoc = myDoc.flatMap (lambda x : ((j, 1) for j in regex.sub(' ', x).lower().split()))

    # This will give us a set of (word, (dictionaryPos, 1)) pairs
    allDictionaryWordsInThatDoc = dictionary.join (wordsInThatDoc).map (lambda x: (x[1][1], x[1][0])).groupByKey()

    # Get tf array for the input string
    myArray = buildArray (allDictionaryWordsInThatDoc.top (1)[0][1])

    myArray1 = myArray[myArray != 0]

    #idf = np.log(np.divide(np.full(20000, numberOfDocs), myArray))

    multiple = np.array([(1/i) if i !=0 else i for i in myArray])
    idfArr = np.log(multiple, where= multiple > 0)

    # Get the tf * idf array for the input string
    myArray = np.multiply (myArray, idf)

    #myArray_total = list(myArray) + [0]*(20000-len(myArray))
    
    #allDocsAsNumpyArraysTFidf
    # Get the distance from the input text string to all database documents, using cosine similarity (np.dot() )
    distances = featuresRDD.map (lambda x : (x[0], np.dot (x[1], myArray)))
    # distances = allDocsAsNumpyArraysTFidf.map (lambda x : (x[0], cousinSim (x[1],myArray)))
    # get the top k distances
    topK = distances.top (k, lambda x : x[1])
    
    # and transform the top k distances into a set of (docID, 1) pairs
    docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))

    # now, for each docID, get the count of the number of times this document ID appeared in the top k
    numTimes = docIDRepresented.reduceByKey(lambda x, y: x+y)
    
    # Return the top 1 of them.
    # Ask yourself: Why we are using twice top() operation here?
    return numTimes.top(k, lambda x: x[1])

In [100]:
print(getPrediction('Sport Basketball Volleyball Soccer', 10))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in multiply


[('CS1_maint:_archived_copy_as_title', 2), ('All_articles_with_unsourced_statements', 1), ('CS1:_long_volume_value', 1), ('Continued_fractions', 1), ('CS1_Spanish-language_sources_(es)', 1), ('Lists_of_states_of_Mexico', 1), ('Articles_with_inconsistent_citation_formats', 1), ('Articles_with_unsourced_statements_from_February_2013', 1), ('Gamma_and_related_functions', 1)]


In [93]:
print(getPrediction('What is the capital city of Australia?', 10))

[('All_stub_articles', 1), ('Connecticut_Republicans', 1), ('Members_of_the_United_States_House_of_Representatives_from_Pennsylvania', 1), ('Pennsylvania_Republicans', 1), ('1833_births', 1), ('1919_deaths', 1), ('19th-century_American_politicians', 1), ('Griswold_family', 1), ('Members_of_the_Connecticut_House_of_Representatives', 1), ('Pennsylvania_United_States_Representative_stubs', 1)]


In [89]:
[2, 2] + [0, 0]

[2, 2, 0, 0]

In [90]:
[0]*3

[0, 0, 0]

In [101]:
textInput = "'Sport Basketball Volleyball Soccer"
k = 10

In [103]:
myDoc = sc.parallelize (('', textInput))
wordsInThatDoc = myDoc.flatMap (lambda x : ((j, 1) for j in regex.sub(' ', x).lower().split()))

In [105]:
wordsInThatDoc.collect()

[('sport', 1), ('basketball', 1), ('volleyball', 1), ('soccer', 1)]

In [107]:
allDictionaryWordsInThatDoc = dictionary.join (wordsInThatDoc).map (lambda x: (x[1][1], x[1][0])).groupByKey ()


In [117]:
allDictionaryWordsInThatDoc.take(2)

[(1, <pyspark.resultiterable.ResultIterable at 0x7fa12cd3c910>)]

In [109]:
allDictionaryWordsInThatDoc.collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x7fa12c9a6f50>)]

In [129]:
myArray = buildArray (allDictionaryWordsInThatDoc.top (1)[0][1])

In [130]:
len(myArray)

20000

In [131]:
myArray

array([0., 0., 0., ..., 0., 0., 0.])

In [132]:
multiple = np.array([(1/i) if i !=0 else i for i in myArray])

In [133]:
multiple

array([0., 0., 0., ..., 0., 0., 0.])

In [167]:
idfArr = np.log(multiplier, myArray, where= myArray > 0)

In [172]:
idfArr

array([0., 0., 0., ..., 0., 0., 0.])

In [136]:
myArray = np.multiply(myArray, idfArr)

In [142]:
distances = featuresRDD.map (lambda x : (x[0], np.dot (x[1], myArray)))

In [143]:
distances.take(2)

[('All_articles_with_unsourced_statements', nan),
 ('Articles_with_inconsistent_citation_formats', nan)]

In [144]:
topK = distances.top (k, lambda x : x[1])

In [146]:
topK[3]

('CS1:_long_volume_value', nan)

In [147]:
docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))


In [148]:
numTimes = docIDRepresented.reduceByKey(lambda x, y: x+y)

In [150]:
numTimes.top(k, lambda x: x[1])

[('CS1_maint:_archived_copy_as_title', 2),
 ('All_articles_with_unsourced_statements', 1),
 ('CS1:_long_volume_value', 1),
 ('Continued_fractions', 1),
 ('CS1_Spanish-language_sources_(es)', 1),
 ('Lists_of_states_of_Mexico', 1),
 ('Articles_with_inconsistent_citation_formats', 1),
 ('Articles_with_unsourced_statements_from_February_2013', 1),
 ('Gamma_and_related_functions', 1)]